#### `pandas` intro here

In [ ]:
import pandas as pd

In [ ]:
migration_errors = pd.read_csv('items_technical_migration.csv')

#### cleaning up column names from Analytics

In [ ]:
migration_errors.columns = [column.lower().replace(' ', '_') for column in migration_errors.columns]

In [ ]:
columns = list(migration_errors.columns)
columns[7] = 'suppressed_holdings'
migration_errors.columns = columns

In [ ]:
migration_errors = migration_errors.rename(columns={'item_id': 'item_pid'})

#### transforming values 

In [ ]:
migration_errors.suppressed_holdings = migration_errors.suppressed_holdings.map({0: 'No', 1: 'Yes'})

#### using `groupby` for analysis

In [ ]:
migration_errors.groupby('location_name').item_pid.count().sort_values(ascending=False)

In [ ]:
call_nos = migration_errors.permanent_call_number.str.extract('([A-Z]{1,2})', expand=False)

In [ ]:
migration_errors.groupby(call_nos).item_pid.count()

#### filtering a `DataFrame` 

In [ ]:
migration_errors = migration_errors.loc[(migration_errors.suppressed_holdings == 'No') & (migration_errors.suppressed_from_discovery == 'No')]

In [ ]:
migration_errors = migration_errors.loc[migration_errors.temporary_location_name == 'None']

In [ ]:
art_books = migration_errors.loc[migration_errors.permanent_call_number.str.startswith('N')]

In [ ]:
art_books.to_csv('./art_books.csv', index=False)

#### using `asyncio`,  `async def`, and `aiohttp`

In [ ]:
import asyncio
import aiohttp
import requests
import time

In [ ]:
test_url = 'http://slowwly.robertomurray.co.uk/delay/1000/url/http://www.cnn.com'

In [ ]:
%time resp = requests.get(test_url)

In [ ]:
def sync_requests(url, n):
    for i in range(n):
        resp = requests.get(url)
        print(f'Made request {i+1}')
        resp.raise_for_status()

In [ ]:
%time sync_requests(test_url, 5)

In [ ]:
async def async_request(url, client, i):
    async with client.get(url) as session:
        await session.text()
        session.raise_for_status()
        print(f'Made request {i+1}')
async def main(url, n):
    async with aiohttp.ClientSession() as client:
        awaitables = [async_request(url, client, i) for i in range(n)]
        await asyncio.gather(*awaitables)

In [51]:
s = time.perf_counter()
await main(test_url, 5)
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")

Made request 5
Made request 2
Made request 3
Made request 1
Made request 4
Script executed in 2.65 seconds.


#### using `AlmaBatch`

In [ ]:
import sys
sys.path.append('/Users/dsmith/Documents/code/alma_batch')
from alma_batch import AlmaBatch
CONFIG_PATH = './test_config.yml'

In [ ]:
ab = AlmaBatch(CONFIG_PATH)

In [ ]:
ab.data = art_books.iloc[:1000].copy()

In [ ]:
ab.validate_data()

In [ ]:
await ab.make_requests_async(batch_size=500, serialize_return=True)

In [ ]:
len(ab.errors)

In [ ]:
ab.errors[0]